In [1]:
import numpy as np
import os
import time

In [2]:
#reads fname, returns a)list of params, b)status: cars, rides, reward
#note that rides are enriched with a final flag (run_to_assign)
def read_input(fname):
    raw_data = open(fname, 'r').read().split('\n')
    [nrow, ncol, ncar, nride, ontime_bonus, max_time] = np.array([int(x) for x in raw_data[0].split(' ')])
    cars = np.array([[i, 0, 0, 0] for i in range(ncar)])
    rides = np.array([[int(x) for x in line.split(' ')] + [True] for line in raw_data[1:-1]])
    return [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, rides, 0]

In [3]:
#manhattan
def distance(starts, stop):
    return abs(starts[:, 0] - stop[0]) + abs(starts[:, 1] - stop[1])

In [197]:
#returns ranking, arrival time, local_reward
#ranking: heuristic to select a ride-car pair to assign
def compute_cost(ride, cars, ontime_bonus, max_time):
    if ride[6]:
        distance_to_start = distance(cars[:, 1:3], ride[:2])
        ride_distance = distance(np.array([ride[:2]]), ride[2:4])
        waiting_time = np.maximum(0, ride[4] - (cars[:, 3] + distance_to_start))
        bonus = ontime_bonus * ((ride[4] - (cars[:, 3] + distance_to_start)) >= 0)
        final_time = cars[:, 3] + distance_to_start + waiting_time + ride_distance

        costs = np.zeros((3, cars.shape[0]))   
        #these car-ride pairs can't be allocated, returning default low ranking
        costs[:, np.logical_or(final_time > max_time, final_time >= ride[5]) == True] = np.array([[-max_time], [-1], [0]])
        #these car-ride pairs can be allocated
        tmp = np.array([np.add(-distance_to_start - waiting_time + bonus, ride_distance), final_time, np.add(bonus, ride_distance)])
        tmp = tmp[:, np.logical_or(final_time > max_time, final_time >= ride[5]) == False]
        costs[:, np.logical_or(final_time > max_time, final_time >= ride[5]) == False] = tmp

        return costs
    else:
        return np.repeat([[-max_time], [-1], [0]], cars.shape[0], axis=1)

def compute_cost2(ride, car, ontime_bonus, max_time):
    if ride[6]:
        distance_to_start = distance(np.array([car[1:3]]), ride[:2])
        ride_distance = distance(np.array([ride[:2]]), ride[2:4])
        waiting_time = np.maximum(0, ride[4] - (car[3] + distance_to_start))
        bonus = ontime_bonus * ((ride[4] - (car[3] + distance_to_start)) >= 0)
        final_time = car[3] + distance_to_start + waiting_time + ride_distance

        if final_time > max_time or final_time >= ride[5]:
            #this car-ride pair can't be allocated, returning default low ranking
            return -max_time, -1, 0
        else:
            return np.add(-distance_to_start - waiting_time + bonus, ride_distance), final_time, np.add(bonus, ride_distance)
    else:
        return -max_time, -1, 0

In [187]:
#create the initial raking matrix - which will be locally updated after each assignment
def init(reward, rides, cars, ontime_bonus, max_time):
    ranks = np.zeros([len(rides), len(cars)])

    for ir, r in enumerate(rides):
        costs = compute_cost(r, cars, ontime_bonus, max_time)
        ranks[ir] = costs[0]

    return ranks

In [202]:
#update the rankings, greedily pick the best pairing, update state
def smart_step(ranks, reward, rides, cars, ontime_bonus, max_time, last_car_id, last_ride_id):
    if(last_car_id != -1):

        #rule out the already allocated run
        ranks[last_ride_id, :] = -max_time
        
        #update the ranking column for the car that was just 'used'
        for ir, r in enumerate(rides):
            ranks[ir][last_car_id] = compute_cost2(r, cars[last_car_id], ontime_bonus, max_time)[0]

    bride, bcar = divmod(np.argmax(ranks), cars.shape[0])

    if ranks[bride, bcar] == -max_time:
        #out of allocable rides
        return [None, None, None, None, -1, -1]

    #now I have the best greedy option, mark the ride as allocated
    _, arrival_time, local_reward = compute_cost2(rides[bride], cars[bcar], ontime_bonus, max_time)
    cars[bcar] = np.array([cars[bcar, 0], rides[bride, 2], rides[bride, 3], arrival_time])
    reward += local_reward 
    rides[bride, 6] = False
    
    return ranks, reward, rides, cars, bcar, bride

In [203]:
def solve_problem(fname):
    tick = time.time()
    print(fname, "started at", time.strftime("%H:%M:%S", time.gmtime(tick)))

    [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, rides, reward] = read_input(fname)

    ranks = init(reward, rides, cars, ontime_bonus, max_time)
    best_car, best_ride, prev_rew = -1, -1, -1
    results = {}
    
    while(True):
        ranks, reward, rides, cars, best_car, best_ride = smart_step(ranks, reward, rides, cars, ontime_bonus, max_time, best_car, best_ride)

        if best_ride == -1:
            break

        prev_rew = reward
        results.setdefault(best_car, []).append(best_ride)

    fname_out = open(fname[:-3] + '.out', 'w')
    for k, v in results.items():
        fname_out.write(str(len(v)) + ' ' + ' '.join([str(i) for i in v]) + '\n')
    for i in range(len(results.keys()), ncar):
        fname_out.write('0\n')

    print('solved', fname, 'with reward of', prev_rew, 'and time', time.strftime("%H:%M:%S", time.gmtime(time.time() - tick)))

In [204]:
def check_solutions(fname):
    [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, rides, reward] = read_input(fname)

    out_fname = fname.replace('.in', '.out')
    raw_data = open(out_fname,'r').read().split('\n')
    
    results = [[int(ride) for ride in car.split(' ')[1:]] for car in raw_data[:-1]]

    total_reward = 0
    for id_car, car_rides in enumerate(results):
        pos = [0, 0]
        reward = 0
        time = 0
        for r in car_rides:
            #print("Car does ride", r)
            time += distance(np.array([pos]), rides[r, 0:2])
            #print("Car moves to ride at time {} ### ride starts at {} and should finish at {}".format(time, rides[r, 4], rides[r, 5]))
            if time <= rides[r, 4]:
                reward += ontime_bonus
                #print("Bonus! Car waits till", rides[r, 4])
            time = max(rides[r, 4], time)
            ride_distance = distance(np.array([rides[r, :2]]), rides[r, 2:4])
            reward += ride_distance
            time += ride_distance
            pos = rides[r, 2:4]
            #print("Car does the ride to time {} ### ride reward {}, total reward {}".format(time, ride_distance, reward))
            assert(time <= rides[r, 5])
        total_reward += reward
        print("{} Car reward {}, partial reward {}".format(id_car, reward, total_reward))
    print(total_reward)


In [205]:
#Just for quick notebook-testing
def get_inputs():
    folder = './data/'

    return [folder + x for x in sorted(os.listdir(folder)) if x[-3:] == '.in']

for fname in get_inputs()[1:3]:
    solve_problem(fname)
    check_solutions(fname)    

./data/b_should_be_easy.in started at 23:36:08
solved ./data/b_should_be_easy.in with reward of [173027] and time 00:00:01
0 Car reward [19118], partial reward [19118]
1 Car reward [16959], partial reward [36077]
2 Car reward [17585], partial reward [53662]
3 Car reward [17647], partial reward [71309]
4 Car reward [14706], partial reward [86015]
5 Car reward [14721], partial reward [100736]
6 Car reward [13749], partial reward [114485]
7 Car reward [12610], partial reward [127095]
8 Car reward [12423], partial reward [139518]
9 Car reward [6099], partial reward [145617]
10 Car reward [10244], partial reward [155861]
11 Car reward [8014], partial reward [163875]
12 Car reward [4100], partial reward [167975]
13 Car reward [3262], partial reward [171237]
14 Car reward [1455], partial reward [172692]
15 Car reward [252], partial reward [172944]
16 Car reward [83], partial reward [173027]
17 Car reward 0, partial reward [173027]
18 Car reward 0, partial reward [173027]
19 Car reward 0, part

KeyboardInterrupt: 